Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC3641 - Aprendizaje Automático Basado en Grafos <br>
Segundo Semestre 2025<br>


<h1><center> Tarea 3  </center></h1>
        Profesor: Marcelo Mendoza<br>
        Fecha de entrega: 22 de octubre


---

## Indicaciones

Se debe entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se asignará puntaje por el código implementado y los comentarios asociados a resultados.
- El notebook debe tener todas las celdas de código ejecutadas.
- Cualquier instancia de copia resultará en un 1.1 como nota de curso.

---

# Integrantes del grupo

* Estudiante 1: Juan Hernandez
* Estudiante 2: Ignacio Vergara
* Estudiante 3: Diego Larraguibel

# Librerías

In [21]:
import dgl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import networkx as nx
import torch

device = "mps"

# Parte 1: Relational Graph Convolutional Network (R-GCN) (20 puntos)

En esta primera sección se trabaja con el dataset BGSDataset, disponible en dgl.

Ver enlace: https://www.dgl.ai/dgl_docs/generated/dgl.data.BGSDataset.html#dgl.data.BGSDataset

**Observación**

Para trabajar con el código visto en clases deben ajustar la versión de pytorch.

## 1.1 Conceptos básicos (5 puntos)


Responda las siguientes preguntas:

1. Mencione y describa los elementos que diferencian las arquitecturas **GCN** y **R-GCN**.

2. Explique las principales diferencias en el proceso de entrenamiento de arquitecturas basadas en **R-GCN**, considerando las tareas **Clasificación de nodos** y **Predicción de enlaces**

Respuesta:

## 1.2 Análisis descriptivo (5 puntos)

Grafique el grafo y calcule medidas descriptivas para caracterizarlo. Comente sus resultados.

**Observación**

En caso de no poder graficar la red completa, se recomienda trabajar con un subconjunto de 15.000 nodos (demora cerca de 12 minutos). Esto **SOLO** aplica para este punto. **La actividad 1.3 debe considerar todo el grafo.**

Respuesta:

In [9]:
ds = dgl.data.BGSDataset()
g = ds[0]
category = ds.predict_category
num_classes = ds.num_classes
train_mask = g.nodes[category].data['train_mask']
test_mask = g.nodes[category].data['test_mask']
label = g.nodes[category].data['label']

Done loading data from cached files.


In [23]:
G = dgl.to_networkx(g)
G_undirected = G.to_undirected()
largest_cc = max(nx.connected_components(G_undirected), key=len)
G_largest = G_undirected.subgraph(largest_cc).copy()
pos_data = nx.spring_layout(G_largest, seed=1001)
plt.figure(figsize=(8, 8))
nx.draw(
    G_largest, pos_data,
    node_size=10,
    node_color='skyblue',
    edge_color='gray',
    with_labels=False,
)
plt.title("Mayor componente conexa del grafo")
plt.show()


KeyboardInterrupt: 

In [ ]:
def funcion_descriptora(G, nombre):
    # Printeo de las características solicitadas
    df_datos = pd.DataFrame([{"Nodos": None, "Aristas": None, "Componentes": None, "Grado Promedio": None, "Diámetro": None, "Camino mínimo promedio": None}])
    df_datos["Nodos"] = len(G.nodes())
    df_datos["Aristas"] = len(G.edges())
    df_datos["Componentes"] = nx.number_connected_components(G)
    degrees = np.array([G.degree(n) for n in G.nodes()])
    df_datos["Grado Promedio"] = np.mean(degrees)
    # En caso de que el grafo no sea conexo se debe determinar el diámetro analizando cada subgrafo del mismo
    if nx.is_connected(G):
        df_datos["Diámetro"] = nx.diameter(G)
    else:
        diam = 0
        for component in nx.connected_components(G):
            subgraph = G.subgraph(component)
            d = nx.diameter(subgraph)
            if diam <= d:
                diam = d
        df_datos["Diámetro"] = diam

    componentes = list(nx.connected_components(G))
    nodos_mayor_comp = max(componentes, key=len)
    mayor_comp = G.subgraph(nodos_mayor_comp)
    cmp_mayor_comp = nx.average_shortest_path_length(mayor_comp) # Camino mínimo promedio de la mayor componente
    df_datos["Camino mínimo promedio"] = cmp_mayor_comp

    print(f"Las estadísticas del grafo {nombre} son:")
    display(df_datos)

funcion_descriptora(G, "BGS")

### Comentario sobre el grafo

## 1.3 Implementación R-GCN (10 puntos)

Entrene una arquitectura R-GCN para clasificar entidades. Defina el número de épocas de manera que se garantice la convergencia del entrenamiento y justifique su elección de hiperpámetros.

Grafique la función de pérdida y accuracy. Comente sus resultados.

Respuesta:

In [ ]:
# Copiamos las clases para construir R-GCN desde la clase 10.
from functools import partial

import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph


class RGCNLayer(nn.Module):
    def __init__(
        self,
        in_feat,
        out_feat,
        num_rels,
        num_bases=-1, # esto indica que num_bases = num_rels
        bias=None,
        activation=None,
        is_input_layer=False,
    ):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases # implementación eficiente de relaciones con combinación lineal de matrices base.
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels
        
        self.weight = nn.Parameter(
            torch.Tensor(self.num_bases, self.in_feat, self.out_feat)
        )
        if self.num_bases < self.num_rels:
            
            self.w_comp = nn.Parameter(
                torch.Tensor(self.num_rels, self.num_bases)
            )
        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))
        # init trainable parameters
        nn.init.xavier_uniform_(
            self.weight, gain=nn.init.calculate_gain("relu")
        )
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(
                self.w_comp, gain=nn.init.calculate_gain("relu")
            )
        if self.bias:
            nn.init.xavier_uniform_(
                self.bias, gain=nn.init.calculate_gain("relu")
            )

    def forward(self, g):
        if self.num_bases < self.num_rels:
            weight = self.weight.view(
                self.in_feat, self.num_bases, self.out_feat
            )
            weight = torch.matmul(self.w_comp, weight).view(
                self.num_rels, self.in_feat, self.out_feat
            )
        else:
            weight = self.weight
        if self.is_input_layer:

            def message_func(edges):
                embed = weight.view(-1, self.out_feat)
                index = edges.data[dgl.ETYPE] * self.in_feat + edges.src["id"]
                return {"msg": embed[index] * edges.data["norm"]}

        else:

            def message_func(edges):
                w = weight[edges.data[dgl.ETYPE]]
                msg = torch.bmm(edges.src["h"].unsqueeze(1), w).squeeze()
                msg = msg * edges.data["norm"]
                return {"msg": msg}

        def apply_func(nodes):
            h = nodes.data["h"]
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {"h": h}

        g.update_all(message_func, fn.sum(msg="msg", out="h"), apply_func)
        

class Model(nn.Module):
    def __init__(
        self,
        num_nodes,
        h_dim,
        out_dim,
        num_rels,
        num_bases=-1,
        num_hidden_layers=1,
    ):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        features = torch.arange(self.num_nodes)
        return features

    def build_input_layer(self):
        return RGCNLayer(
            self.num_nodes,
            self.h_dim,
            self.num_rels,
            self.num_bases,
            activation=F.relu,
            is_input_layer=True,
        )

    def build_hidden_layer(self):
        return RGCNLayer(
            self.h_dim,
            self.h_dim,
            self.num_rels,
            self.num_bases,
            activation=F.relu,
        )

    def build_output_layer(self):
        return RGCNLayer(
            self.h_dim,
            self.out_dim,
            self.num_rels,
            self.num_bases,
            activation=partial(F.softmax, dim=1),
        )

    def forward(self, g):
        if self.features is not None:
            g.ndata["id"] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop("h")

**Observación**

Para la pregunta 2 se recomienda revisar la publicación **Modeling
relational data with graph convolutional networks** (Schlichtkrull et al., 2018). Disponible en: https://arxiv.org/abs/1703.06103

# Parte 2: Graph Transformer (20 puntos)

En esta segunda sección se debe trabajar con el conjunto de datos **caco2_wang**, contenido en el benchmark de ADMET.

**Observación**

Para cargar el dataset debe seguir el procedimiento visto en clases (8 - Graph-transformer)

## 2.1 Conceptos básicos (5 puntos)

Responda las siguientes preguntas:

1. En el contexto de Graph Transformers, defina el concepto de **sparseness**. ¿Qué implicancias tiene este concepto en las capacidades de cómputo requeridas para trabajar con dichas arquitecturas?

2. Explique el concepto de **positional encoding** en el contexto de los Graph Transformers y compare su formulación con la empleada en el Transformer original (Vaswani et al., 2017).

**Observación**

Se recomienda revisar la publicación **Attention Is All You Need** (Vaswani et al., 2017). Disponible en: https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf

Respuesta:

## 2.2 Implementación Graph Transformer (15 puntos)

Proponga cuatro modelos para la tarea de regresión asociada al dataset **caco2_wang**, utilizando la arquitectura **Graph Transformer**. Para ello, haga modificaciones tanto en el número de cabezales como en el número de capas. Justifique sus decisiones.

Compare sus resultados con el Leaderboard disponible en: https://tdcommons.ai/benchmark/admet_group/01caco2/

**Observación**

* Defina el número de épocas de manera que se garantice la convergencia del entrenamiento.
* Trabaje con los conjuntos ya definidos de train, val y test.

Respuesta:

# Parte 3: Graphormer (20 puntos)

En esta tercera sección se debe trabajar con el dataset **PubMed**.

Este conjunto de datos está disponibles en pytorch-geometric. Ver enlace: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.Planetoid.html#torch_geometric.datasets.Planetoid

A continuación, se presenta el código para cargar el dataset.

In [3]:
from torch_geometric.datasets import Planetoid

In [4]:
dataset = Planetoid('./data/PubMed', 'PubMed') #definir root y el dataset que desea descargar
data = dataset[0]

Processing...
Done!


## 3.1 Conceptos básicos (5 puntos)

Responda las siguientes preguntas:
1. ¿Que limitaciones del **Graph Attention Network** (GAT) intenta superar el Graphormer?
2. Describa las principales diferencias entre **Graphormer** y el **Graph Transformer original**.
3. Explique el concepto de **pooling** en el contexto de grafos. ¿En qué se diferencian **DiffPool** de **MinCutPool**?

Respuesta:

## 3.2 DiffPool (5 puntos)

En esta sección se entrenará un modelo Graphormer utilizando el método de pooling jerárquico DiffPool. Para ello, siga los siguientes pasos:
1. Entrene un Graphormer con pooling jerárquico `DiffPool` con 300 épocas.
2. Por cada 20 épocas, reporte los siguientes indicadores:
    - Pérdida de clasificación
    - Regularización
    - Accuracy del test
    - NMI y ARI
    - Promedio tamaño de clusters ± desviación estandar
3. Entrene t-SNE sobre los nodos en el *embedding* final y sobre los centroides, y grafique los resultados. Finalmente, comente sus resultados.

Respuesta:

## 3.3 MinCutPool (5 puntos)

En esta sección se entrenará un modelo Graphormer utilizando el método de pooling jerárquico MinCut. Para ello, siga los siguientes pasos:
1. Entrene un Graphormer con pooling jerárquico `MinCutPool` con 300 épocas.
2. Por cada 20 épocas, reporte los siguientes indicadores:
    - Pérdida de clasificación
    - Regularización
    - Accuracy del test
    - NMI y ARI
    - Promedio tamaño de clusters ± desviación estandar
3. Entrene t-SNE sobre los nodos en el *embedding* final y sobre los centroides, y grafique los resultados. Finalmente, comente sus resultados.

Respuesta:

## 3.4 Conclusiones: DiffPool vs MinCutPool (5 puntos)

* ¿Qué modelo obtuvo mejor accuracy, NMI o ARI?

* ¿Qué diferencias se observaron en la convergencia de las pérdidas entre DiffPool y MinCutPool?

* ¿Hubo una diferencia clara en la capacidad de los modelos para formar clusters coherentes?

* ¿Cuál método mostró una mejor separación de clases en los gráficos t-SNE?

* ¿Cuál de los métodos recomendarías para este tipo de tarea y por qué?

Respuesta: